# 4.2 Laser Scanners

One of the most widely used types of range/bearing sensors is the 2D LiDAR (*Light-based Detection And Ranging*) or laser scanner. Their principle of operation is simple: the sensor emits a pulse of light of a particular frequency, and waits for it to be reflected back by the environment. Once the reflected pulse is received, the distance it has traveled can be calculated from the time-of-flight (TOF).


A 2D scanner repeats this operation many times while rotating around its own center, thus obtaining information about the distance to the environment at different angles on its plane of operation. The main parameters that describe a laser scanner are:

- The angle increment between one measurement and the next is the **angular resolution** of the scanner, and determines how detailed the complete measurement is.
- The **field of view (FOV)** determines the angle limits of where the scan starts and stops. While some commercial LiDARs are capable of scanning all around themselves, it is often necessary to limit the FOV due to where the sensor is mounted on the robot (as we want to prevent the sensor from observing the body of the robot itself).
- The **max distance**, past which the sensor is not able to produce a measurement at all.
- The **noise**, of course. You know how this goes by now: sensor measurements are never fully reliable. Depending on the specifics of the sensor, the uncertainty of the measurements might only be reported for the *distance* reading (that is, a scalar **variance**), or for both the distance and the angle (a **covariance matrix**).  

<figure style="text-align:center">
    <img src="images/sick_and_hokuyo_laser_scanners.png" width="600px" alt="">
    <figcaption>Top row, 2D laser scanners from <a href="https://www.sick.com/es/es/catalog/productos/safety/escaner-laser-de-seguridad/c/g569359" href="_blank">SICK</a>. Bottom row, left, 2D laser scanners from <a href="https://www.hokuyo-aut.jp/search/?cate01=1&cate02=1&cate03=" target="_blank">Hokuyo</a>, right, illustration of the field of view of some models of this brand.</figcaption>
</figure>

Let's create a simulated Laser Scanner with these parameters and test it out!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets

import sys
sys.path.append("..")
from utils.laser.laser2D import Laser2D
from utils.DrawRobot import DrawRobot
from utils.tcomp import tcomp

### Creating the scanner and the map

We will use the `Laser2D` class imported above. The constructor of this class expects the parameters we just discussed:
- `FOV` -- Sensor field of view (radians)
- `resolution` -- Sensor resolution (radians)
- `max_distance` -- Max operating distance of the sensor (meters)
- `noise_cov` -- covariance matrix characterizing sensor noise
- `pose` -- sensor pose (column vector with x and y positions, and orientation theta)

The following function shows you how to create an instance of the `Laser2D` class:

In [ ]:
def create_laser_at_origin(FOV, resolution, max_distance, noise_cov):
    origin_pose = np.vstack([0, 0, 0])
    laser = Laser2D(FOV, resolution, max_distance, noise_cov, origin_pose)
    return laser

The `Laser2D` class has two important methods: `set_pose()`, to tell it where to take an observation from; and `take_observation()`, which does exactly what you think. However, you might be asking "an observation of *what*, exactly? We haven't told the laser what the environment is like." Well, you're right, so let's do just that!

We will define the map as a list of polygonal **contours**. In the following cell, you will see an example:

In [ ]:
def create_room_map():
    # Define the environment
    # A polygon contour is defined as two arrays, specifying the x and y coordinates of each point
    # The polygon is generated by connecting (x_t, y_t) to (x_t+1, y_t+1) with a straight line segment

    # NaN denotes the end of each polygon
    walls = np.array([[2, 12, 12, 10, 10, 9.5, 9.5, 2, 2, np.nan],
                        [2, 2, 10, 10, 8, 8, 10, 10, 2, np.nan]])

    kitchen = np.array([[2, 8.5, 8.5, 9.5, 9.5, 2, 2, np.nan],
                        [9, 9, 8, 8, 10, 10, 9, np.nan]])

    sofa = np.array([[3.5, 7, 7, 3.5, 3.5, np.nan],
                        [6, 6, 7, 7, 6, np.nan]])

    tv_cabinet = np.array([[2.5, 7.5, 7.5, 2.5, 2.5, np.nan],
                            [2, 2, 3, 3, 2, np.nan]])

    table = np.array([[9.5, 11, 11, 9.5, 9.5, np.nan],
                        [3.5, 3.5, 6, 6, 3.5, np.nan]])


    virtual_map = np.concatenate([walls, kitchen, sofa, tv_cabinet, table], axis=1)
    return virtual_map

virtual_map = create_room_map() 
# Print map shape
print('Map shape:', virtual_map.shape)

The `plot_virtual_map()` function is provided to help you visually interpret the process and understand what is happening:

In [ ]:
def plot_virtual_map(virtual_map, dimensions_only = False):
    """
    Plots the robot pose, virtual map, beam endpoints, and optionally grid cells to be updated.
    
    Parameters:
    - virtual_map: environment representation using lines.
    """
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot the virtual map (environment boundaries or walls)
    if not dimensions_only:
        plt.plot(virtual_map[0, :], virtual_map[1, :], 'k-')    
    
    # Set grid and axis limits
    plt.grid()
    plt.xlim([np.nanmin(virtual_map[0])-2,np.nanmax(virtual_map[0])+2]) # nanmin ignores nan numbers
    plt.ylim([np.nanmin(virtual_map[1])-2,np.nanmax(virtual_map[1])+2])  
    
    # Title and axis labels
    plt.title('Living-room Map')
    plt.xlabel('X position (m)')
    plt.ylabel('Y position (m)')

    return fig, ax

plot_virtual_map(virtual_map);

### **<span style="color:green"><b><i>ASSIGNMENT 1: Taking an observation</i></b></span>** 

To use the scanner, we will call the `take_observation()` method, passing it the map as an argument.

Similarly to the landmark-based sensors we saw in the last notebook, our laser scanner will produce observations as angle-distance pairs. In other words, the measurements are in *polar coordinates*. Remember that you can express a sensor measurement given in polar coordinates ($z_p=[r,\alpha]^T$) as cartesian coordinates ($z_c=[z_x,z_y]^T)$ with the following expression:
 
 $$
     z_c = \begin{bmatrix} z_x \\ z_y \end{bmatrix} 
         = \begin{bmatrix} r \ cos\alpha \\ r \ sin\alpha \end{bmatrix} 
 $$ 

Remember also that all the observations are taken from the point of view of the sensor, and thus are expressed in its local system of reference. To express the measurements in world space (on the map), you can use our old trick: **pose composition**.

Let's put all this into practice. Implement a function which takes a sensor reading $z$ (which is a **list** of points, in polar coordinates and relative to the sensor pose), and plots those points on the map as a *point cloud*.

In [ ]:
def plot_observations(z, pose, fig, ax, draw_lines = True, point_color = 'r'):
    """Draw a sensor observation taken from a given pose on a figure.

    Keyword arguments:
    z -- Laser observation
    pose -- Pose from which the observation was taken
    fig, ax -- Figure to plot in
    """
    for i in range (z.shape[1]):
        distance = None
        angle = None

        # calculate the cartesian coordinates of the observed point in local space
        x_local = None
        y_local = None

        # compose with the robot pose to express the point in global space
        pose_obs_global = tcomp(None, None)
        x = pose_obs_global[0]
        y = pose_obs_global[1]

        if draw_lines:
            ax.plot([x, pose[0]], [y, pose[1]], 'g--', linewidth=1)
        ax.plot(None, None, 'o', markersize=4, color=point_color)

Use the following code to test it out:

In [ ]:
def run_laser_and_plot(laser, pose, fig, ax, draw_lines = True, color='r'):
    laser.set_pose(pose)

    # take the measurement
    measurement = laser.take_observation(virtual_map)

    # Plot!
    DrawRobot(fig, ax, pose, axis_percent=0.015, color=color, linewidth=1.5);
    plot_observations(measurement, pose, fig, ax, draw_lines, color)
    return fig, ax


def test_laser(FOV, resolution, max_distance, distance_noise, angle_noise):
    laser = create_laser_at_origin(FOV, resolution, max_distance, np.diag([distance_noise, angle_noise]))

    fig, ax = plot_virtual_map(virtual_map);
    run_laser_and_plot(laser, np.vstack([6, 4, np.pi/2]), fig, ax);

ipywidgets.interact(
    test_laser,
    FOV=ipywidgets.FloatSlider(3 * np.pi/2, min=0.2, max=2 * np.pi),
    resolution=ipywidgets.FloatSlider(0.2, min=0.02, max=0.5, step=0.01),
    max_distance=ipywidgets.FloatSlider(10, min=1, max=10),
    distance_noise=ipywidgets.FloatSlider(0.01, min=0.0, max=0.2, step=0.01),
    angle_noise=ipywidgets.FloatSlider(0.001, min=0, max=0.2, step=0.01)
)

Expected output:

<figure style="">
    <img src="images/assignment_2_1.png" width="600px" alt="">
</figure>

### <font color="blue"><b><i>Thinking about it</i></b></font>

Having completed the code above, you will be able to **answer the following questions**:

- Which has a greater impact on the reliability of the measurements: the variance of the distance reading, or the variance of the angle at which the measurement happens? Which would you guess is larger in real sensors, and why?

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

- Without actually implementing it: how could we calculate the uncertainty about the position of each of the measured points (in cartesian coordinates)? *Hint: we already did this for the landmarks!*

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

- We are assuming we know the pose of the robot, but what if there was also uncertainty associated with it? How do you combine both sources of uncertainty? *Hint: we **also** did this for the landmarks!*

    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

### **<span style="color:green"><b><i>ASSIGNMENT 2: Exploring the environment</i></b></span>** 

Now that we can plot the measurements in global coordinates, we should be able to roughly reconstruct the contours of the map by moving around the environment and repeatedly taking observations from multiple poses. Let's try that out: 

In [ ]:
def explore_environment():
    laser = create_laser_at_origin(FOV = 3 * np.pi/2,
        resolution = 0.05,
        max_distance = 10,
        noise_cov = np.diag([0.03, 0.01]))

    # The robot must make these movements, in this order
    # 0, 0, 0
    # 2, 0, 0
    # 2, 0, 0
    # 0, 2, np.pi/2
    # 2, 0, 0
    # Format them correctly in the following list

    # each pose and its corresponding observations will have a different color
    pose_increments_list = [
        (None, 'orangered'),
        (None, 'olive'),
        (None, 'teal'),
        (None, 'magenta'),
        (None, 'indigo'),
    ]

    fig, ax = plot_virtual_map(virtual_map, True); #create the base figure with the map dimensions, but without the lines

    current_pose = np.vstack([4,4,0]) # starting pose

    # take a step and read the sensor at the new pose
    for step, color in pose_increments_list:
        current_pose = None 
        run_laser_and_plot(None, None, fig, ax, False, color)

explore_environment()

Expected output:

<figure style="">
    <img src="images/assignment_2_2.png" width="600px" alt="">
</figure>

This looks pretty promising! Looking at this image, it is quite easy to get an intuition for the shape of the environment and the presence of obstacles. However, due to the noise in the sensor observations, it is difficult to tell where *exactly* the contours of the map really are. 

This is the problem of **mapping**, which we will talk about in more detail in chapter 6. It probably won't surprise you to learn that, in order to deal with these noisy observations, we will tackle this problem probabilistically. 

An even more complicated version of this process is necessary when the pose of the robot is uncertain, as we can't even reliably place the noisy points on the global map. To solve this (more realistic) case, we would need to simultaneously figure out the correct pose of the robot *and* build the map. This is the wonderfully named problem of **SLAM:** *Simultaneous Localization And Mapping*, which we will talk about in chapter 7. 

## <font color="orange"><b><i>AI Appendix</i></b></font>

### A.1 How I used AI
- **Model/tool:**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **What I asked AI to help with (bullets):**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **Best prompt I used (paste):**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **What I kept vs. changed from the AI output:**
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

### A.2 AI review 
Ask one generative AI to review your memo using this prompt:

```
Act as a technical reviewer of this computer vision lab notebook. Read my memo and return exactly 5 bullet points:

- two on theory (key ideas),
- two on practice (relevant programming concepts of python used, not particular functions or variables),
- one on how to improve the code (most critical aspect: limitations, parameter choices, runtime/robustness, possible bugs).

Keep each bullet ≤15 words and make them specific to my work.
```

Paste its **5** items here:

- **(Theory 1)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Theory 2)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Practice 1)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Practice 2)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
- **(Improvement)** AI claim:
  <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>

Finally, **do you agree with them?**, **would you have selected others?** 
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>